<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/CollabSetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Pyspark


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [72.6 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,688 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB


### Import




In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from pyspark.sql.functions import pandas_udf, PandasUDFType, udf

In [3]:
df = spark.range(0, 10 * 1000 * 1000).withColumn('id', (col('id') / 10000).cast('integer')).withColumn('v', rand())
df.cache()
df.count()

df.show()

+---+--------------------+
| id|                   v|
+---+--------------------+
|  0| 0.48723839870162955|
|  0| 0.15992292148320963|
|  0|  0.7117016419527978|
|  0| 0.32731501110858585|
|  0| 0.06733168598051786|
|  0| 0.44530526631032974|
|  0|  0.2877573197320078|
|  0|  0.2061868747013622|
|  0|  0.5611732422236512|
|  0|  0.5621137614531215|
|  0|  0.4260670334473634|
|  0| 0.14480539521192126|
|  0| 0.04314252400358565|
|  0|0.042197541645179526|
|  0| 0.13284635172585146|
|  0|   0.859677668094348|
|  0|  0.4115068723620233|
|  0|   0.677421189108456|
|  0|  0.6925199283671691|
|  0|  0.6527299352576307|
+---+--------------------+
only showing top 20 rows



In [6]:
@udf('double')
def plus_one(v):
    return v + 1

%timeit df.withColumn('v', plus_one(df.v)).agg(count(col('v'))).show()

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

25.6 s ± 1.68 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
@pandas_udf("double", PandasUDFType.SCALAR)
def pandas_plus_one(v):
    return v + 1

%timeit df.withColumn('v', pandas_plus_one(df.v)).agg(count(col('v'))).show()

/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

6.12 s ± 684 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# Pandas UDF
import pandas as pd
from pyspark.sql.functions import pandas_udf, log2, col

@pandas_udf('long')
def pandas_plus_one(s: pd.Series) -> pd.Series:
    return s + 1

# pandas_plus_one("id") is identically treated as _a SQL expression_ internally.
# Namely, you can combine with other columns, functions and expressions.
spark.range(10).select(
    pandas_plus_one(col("id") - 1) + log2("id") + 1).show()

+--------------------------------------------+
|((pandas_plus_one((id - 1)) + LOG2(id)) + 1)|
+--------------------------------------------+
|                                        NULL|
|                                         2.0|
|                                         4.0|
|                           5.584962500721156|
|                                         7.0|
|                           8.321928094887362|
|                           9.584962500721156|
|                          10.807354922057604|
|                                        12.0|
|                          13.169925001442312|
+--------------------------------------------+



In [9]:
# Pandas Function API
from typing import Iterator
import pandas as pd


def pandas_plus_one(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    return map(lambda v: v + 1, iterator)


# pandas_plus_one is just a regular Python function, and mapInPandas is
# logically treated as _a separate SQL query plan_ instead of a SQL expression.
# Therefore, direct interactions with other expressions are impossible.
spark.range(10).mapInPandas(pandas_plus_one, schema="id long").show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
+---+



### Plane Python UDF

In [13]:
# In Python
from pyspark.sql.types import LongType

# Create cubed function
def cubed(s):
  return s * s * s

# Register UDF
spark.udf.register("cubed_udf", cubed, LongType())

# Generate temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")


spark.sql("SELECT id, cubed_udf(id) AS id_cubed_udf FROM udf_test").show()

+---+------------+
| id|id_cubed_udf|
+---+------------+
|  1|           1|
|  2|           8|
|  3|          27|
|  4|          64|
|  5|         125|
|  6|         216|
|  7|         343|
|  8|         512|
+---+------------+



In [19]:
# In Python
# Import pandas
import pandas as pd

# Import various pyspark SQL functions including pandas_udf
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the cubed function
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

# Create the pandas UDF for the cubed function
cubed_udf_pandas = pandas_udf(cubed, returnType=LongType())


# Create a Pandas Series
x = pd.Series([1, 2, 3])

# The function for a cubed_udf_pandas executed with local Pandas data
print(cubed(x))

0     1
1     8
2    27
dtype: int64


In [18]:
# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.range(1, 4)

# Execute function as a Spark vectorized UDF
df.select("id", cubed_udf_pandas(col("id"))).show()

+---+---------+
| id|cubed(id)|
+---+---------+
|  1|        1|
|  2|        8|
|  3|       27|
+---+---------+



### iterator

In [20]:
import pandas as pd
from typing import Iterator
from pyspark.sql.functions import col, pandas_udf, struct

pdf = pd.DataFrame([1, 2, 3], columns=["x"])
df = spark.createDataFrame(pdf)

# When the UDF is called with the column,
# the input to the underlying function is an iterator of pd.Series.
@pandas_udf("long")
def plus_one(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for x in batch_iter:
        yield x + 1

df.select(plus_one(col("x"))).show()

+-----------+
|plus_one(x)|
+-----------+
|          2|
|          3|
|          4|
+-----------+



In [25]:
y_bc = spark.sparkContext.broadcast(1)

print(y_bc)

@pandas_udf("long")
def plus_y(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    y = y_bc.value  # initialize states
    print(y)
    try:
        for x in batch_iter:
            yield x + y
    finally:
        pass  # release resources here, if any

df.select(plus_y(col("x"))).show()

+---------+
|plus_y(x)|
+---------+
|        2|
|        3|
|        4|
+---------+

